In [3]:
import re
import json
from pathlib import Path

# --- KLASÖR YOLLARI ---
DATA_DIR = r"C:\Users\hukus\Downloads\Code4Her\knowledgw_base"
OUT_FILE = r"C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl"

# --- VERİ AYIKLAMA FONKSİYONU ---
def parse_file(file_path: Path):
    text = file_path.read_text(encoding="utf-8")
    
    # "Soru:" ve "## Soru:" her iki formatı da destekleyen regex
    pattern = re.compile(
        r"(?:##\s*)?Soru[:\-–]\s*(.*?)\s*Cevap[:\-–]\s*(.*?)(?=\n\Soru[:\-–]|\n##\s*Soru[:\-–]|\Z)",
        re.DOTALL | re.IGNORECASE
    )

    qa_pairs = []
    for match in pattern.finditer(text):
        question = match.group(1).strip().replace("\n", " ")
        answer = match.group(2).strip().replace("\n", " ")
        qa_pairs.append({
            "source": file_path.name,
            "question": question,
            "answer": answer
        })
    return qa_pairs

# --- ANA FONKSİYON ---
def main():
    out_path = Path(OUT_FILE)
    data_dir = Path(DATA_DIR)
    
    all_data = []
    print(f"Veri klasörü: {data_dir}")
    print(f"Çıktı dosyası: {out_path}")
    
    for p in data_dir.glob("*.txt"):
        parsed = parse_file(p)
        all_data.extend(parsed)
        print(f"{p.name} dosyasından {len(parsed)} kayıt çıkarıldı.")
    
    # JSONL yaz
    with out_path.open("w", encoding="utf-8") as fout:
        for item in all_data:
            fout.write(json.dumps(item, ensure_ascii=False) + "\n")
    
    print(f"\nToplam {len(all_data)} kayıt {out_path} dosyasına yazıldı.")

# --- ÇALIŞTIR ---
if __name__ == "__main__":
    main()


Veri klasörü: C:\Users\hukus\Downloads\Code4Her\knowledgw_base
Çıktı dosyası: C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl
6284_sayili_kanun.txt dosyasından 124 kayıt çıkarıldı.
gumushane_el_kitabi.txt dosyasından 95 kayıt çıkarıldı.
KADES_bilgilendirme.txt dosyasından 12 kayıt çıkarıldı.
kadin_dayanisma_vakfi_rehber.txt dosyasından 22 kayıt çıkarıldı.
mor_cati_rehber.txt dosyasından 393 kayıt çıkarıldı.
sonim_yonetmelik.txt dosyasından 80 kayıt çıkarıldı.
sıdas.txt dosyasından 34 kayıt çıkarıldı.

Toplam 760 kayıt C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl dosyasına yazıldı.


In [3]:
import json
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Modeli yükle
model = SentenceTransformer("intfloat/multilingual-e5-base")

# Veri dosyasının yolu
input_file = r"C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl"
output_embeddings = r"C:\Users\hukus\Downloads\Code4Her\knowledge_embeddings.npy"
output_texts = r"C:\Users\hukus\Downloads\Code4Her\knowledge_texts.json"

texts = []

# JSONL dosyasını oku
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line)
        texts.append(record.get("text") or record.get("content") or "")

# Embedding üretimi
embeddings = []
for text in tqdm(texts, desc="Embedding üretiliyor..."):
    emb = model.encode(text)
    embeddings.append(emb)

embeddings = np.array(embeddings)

# Embedding'leri ve metinleri kaydet
np.save(output_embeddings, embeddings)

with open(output_texts, "w", encoding="utf-8") as f:
    json.dump(texts, f, ensure_ascii=False, indent=2)

print(f"{len(embeddings)} adet embedding üretildi ve kaydedildi:")
print(f"- Embedding dosyası: {output_embeddings}")
print(f"- Metin dosyası: {output_texts}")


Embedding üretiliyor...: 100%|██████████| 760/760 [00:13<00:00, 55.41it/s]

760 adet embedding üretildi ve kaydedildi:
- Embedding dosyası: C:\Users\hukus\Downloads\Code4Her\knowledge_embeddings.npy
- Metin dosyası: C:\Users\hukus\Downloads\Code4Her\knowledge_texts.json


In [4]:
!pip install faiss-cpu numpy

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Dosya yolları
JSONL_FILE = r"C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl"
EMB_PATH = r"C:\Users\hukus\Downloads\Code4Her\knowledge_embeddings.npy"
TEXT_PATH = r"C:\Users\hukus\Downloads\Code4Her\knowledge_texts.json"

# Modeli yükle
model = SentenceTransformer("intfloat/multilingual-e5-base")

texts = []

# JSONL dosyasını oku ve question+answer'i birleştir
with open(JSONL_FILE, "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line)
        question = record.get("question", "").strip()
        answer = record.get("answer", "").strip()
        if question and answer:
            # Tek bir metin halinde birleştir
            text = f"Soru: {question} Cevap: {answer}"
            texts.append(text)

print(f"Toplam metin: {len(texts)}")
print("Örnek:", texts[:3])

# Embedding üretimi
embeddings = []
print("Embedding üretiliyor...")
for t in tqdm(texts):
    emb = model.encode(t)
    embeddings.append(emb)

embeddings = np.array(embeddings, dtype="float32")

# Dosyaları kaydet
np.save(EMB_PATH, embeddings)
with open(TEXT_PATH, "w", encoding="utf-8") as f:
    json.dump(texts, f, ensure_ascii=False, indent=2)

print(f"\n{len(texts)} adet embedding üretildi ve kaydedildi:")
print(f"- Embedding dosyası: {EMB_PATH}")
print(f"- Metin dosyası: {TEXT_PATH}")


Toplam metin: 760
Örnek: ['Soru: Bu yönetmeliğin amacı nedir? [cite_start] Cevap: Bu yönetmeliğin amacı, şiddete uğrayan veya şiddete uğrama tehlikesi bulunan kişilerin korunması, bu kişilere yönelik şiddetin önlenmesi ve şiddet uygulayanlar hakkında alınacak tedbirlere ilişkin usul ve esasları düzenlemektir[cite: 5].', 'Soru: 6284 sayılı kanun kimleri korur? Sadece kadınlar için mi? Cevap: Hayır, kanun sadece kadınları değil; [cite_start]şiddete uğrayan veya şiddete uğrama tehlikesi bulunan kadınları, çocukları, aile bireylerini ve tek taraflı ısrarlı takip mağduru olan kişileri korur[cite: 5].', 'Soru: Bu kanun şiddet uygulayanlar hakkında da bir düzenleme içeriyor mu? [cite_start] Cevap: Evet, kanun aynı zamanda şiddet uygulayan veya uygulama ihtimali olan kişiler hakkında şiddetin önlenmesine yönelik tedbirleri ve bu tedbirlerin nasıl alınacağını da kapsar[cite: 5].  ---  # Konu: Temel Kavramlar ve Yasal Tanımlar']
Embedding üretiliyor...


100%|██████████| 760/760 [00:35<00:00, 21.56it/s]


760 adet embedding üretildi ve kaydedildi:
- Embedding dosyası: C:\Users\hukus\Downloads\Code4Her\knowledge_embeddings.npy
- Metin dosyası: C:\Users\hukus\Downloads\Code4Her\knowledge_texts.json


In [14]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# Dosya yolları
EMB_PATH = r"C:\Users\hukus\Downloads\Code4Her\knowledge_embeddings.npy"
TEXT_PATH = r"C:\Users\hukus\Downloads\Code4Her\knowledge_texts.json"
FAISS_INDEX_PATH = r"C:\Users\hukus\Downloads\Code4Her\faiss_index.index"

# Embedding ve metinleri yükle
embeddings = np.load(EMB_PATH)
with open(TEXT_PATH, "r", encoding="utf-8") as f:
    texts = json.load(f)

# Embedding boyutu
dimension = embeddings.shape[1]

# FAISS index oluştur (cosine similarity için normalize ediyoruz)
index = faiss.IndexFlatIP(dimension)  # Inner Product
faiss.normalize_L2(embeddings)       # Cosine similarity için normalize
index.add(embeddings)

# Index'i kaydet
faiss.write_index(index, FAISS_INDEX_PATH)
print(f"FAISS index oluşturuldu ve kaydedildi: {FAISS_INDEX_PATH}")

# Arama fonksiyonu
model = SentenceTransformer("intfloat/multilingual-e5-base")

def search_similar(query, top_k=3):
    query_vec = model.encode([query]).astype("float32")
    faiss.normalize_L2(query_vec)
    D, I = index.search(query_vec, top_k)  # D = skorlar, I = indeksler
    print("\n🔍 En benzer kayıtlar:\n")
    for idx, score in zip(I[0], D[0]):
        print(f"Skor: {score:.4f}")
        print(f"Metin: {texts[idx][:500]}...\n")  # İlk 500 karakteri göster

# Test
search_similar("Sığınağa kimler başvurabilir?")


FAISS index oluşturuldu ve kaydedildi: C:\Users\hukus\Downloads\Code4Her\faiss_index.index

🔍 En benzer kayıtlar:

Skor: 0.8858
Metin: Soru: Sığınağa nasıl başvurulur ve başvuruyu kim yapmalıdır? Cevap: Sığınak başvurusunu kendiniz yapmalısınız. Bakanlığa bağlı bir sığınağa yerleşmek için İl Müdürlüğü'ne, ŞÖNİM'e, Polise/Jandarmaya ya da Kaymakamlığa/Valiliğe başvurabilirsiniz....

Skor: 0.8489
Metin: Soru: Sığınağa nasıl giderim? Ulaşım sağlanıyor mu? Cevap: Evet, ŞÖNİM tarafından il içi ve il dışı nakillerde ulaşım için araç tahsis edilir. Ayrıca ulaşım giderleriniz ile diğer zorunlu giderleriniz de karşılanır. Hayati tehlikenizin bulunması halinde ise yerleştirileceğiniz yere kolluk (polis/jandarma) refakat eder....

Skor: 0.8488
Metin: Soru: Bakanlığa bağlı sığınaklara kabul için yaş sınırı nedir? Cevap: Sığınağa kabul edilmek için 18 yaşından büyük, 60 yaşından küçük olmanız gerekir....

